In [1]:
!pip install git+https://github.com/ichironakamoto/Japan.git

  Cloning https://github.com/ichironakamoto/Japan.git to /tmp/pip-req-build-_zgdyd3e
  Running command git clone -q https://github.com/ichironakamoto/Japan.git /tmp/pip-req-build-_zgdyd3e
  Created wheel for covid19-inference: filename=covid19_inference-0.0.10-cp36-none-any.whl size=27625 sha256=77096197bb31a9856d06e90603806fddb13d5d5d3f6dbf992572bdb8912d54ca
  Stored in directory: /tmp/pip-ephem-wheel-cache-qj7qwa8w/wheels/1e/a5/16/7d019c629951d7e8b52c76c794e547a19333680b3c6dbac3bd
Successfully built covid19-inference


In [2]:
import sys
sys.path.append('../..')
import datetime
import pickle

import numpy as np
import pymc3 as pm
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib

try: 
    import covid19_inference as cov19
except ModuleNotFoundError:
    sys.path.append('..')
    import covid19_inference as cov19
    
path_to_save = '../../figures/'
path_save_pickled = '../../data/'
rerun = True

confirmed_cases = cov19.get_jhu_confirmed_cases()

date_data_begin = datetime.datetime(2020,3,1)
# date_data_end   = cov19.get_last_date(confirmed_cases)
date_data_end = datetime.datetime(2020,4,21)
num_days_data = (date_data_end-date_data_begin).days
diff_data_sim = 12 # should be significantly larger than the expected delay, in 
                   # order to always fit the same number of data points.
num_days_future = 21
date_begin_sim = date_data_begin - datetime.timedelta(days = diff_data_sim)
date_end_sim   = date_data_end   + datetime.timedelta(days = num_days_future)
num_days_sim = (date_end_sim-date_begin_sim).days

country = 'Germany'

cases_obs = cov19.filter_one_country(confirmed_cases, country,
                                     date_data_begin, date_data_end)

prior_date_mild_dist_begin =  datetime.datetime(2020,3,9)
prior_date_strong_dist_begin =  datetime.datetime(2020,3,16)
prior_date_contact_ban_begin =  datetime.datetime(2020,3,23)

change_points = [dict(pr_mean_date_begin_transient = prior_date_mild_dist_begin,
                      pr_median_lambda = 1,
                      pr_sigma_lambda = 1),
                 dict(pr_mean_date_begin_transient = prior_date_strong_dist_begin,
                      pr_sigma_date_begin_transient = 1,
                      pr_median_lambda = 0.5,
                      pr_sigma_lambda = 1),
                 dict(pr_mean_date_begin_transient = prior_date_contact_ban_begin,
                      pr_sigma_date_begin_transient = 1,
                      pr_median_lambda = 0.25,
                      pr_sigma_lambda = 1)]

model = cov19.SEIR_with_extensions(np.diff(cases_obs),
                                    change_points,
                                    date_begin_sim,
                                    num_days_sim,
                                    diff_data_sim, 
                                    N=83e6,
                                    priors_dict=None,
                                    with_random_walk=False,
                                    weekends_modulated=True,
                                    weekend_modulation_type = 'abs_sine')

if rerun:
    trace = pm.sample(model=model, init='advi+adapt_diag', n_init=5000,draws=4000, tune=1000)

    pickle.dump([model, trace], open('SEIR.pickled', 'wb'))

else: 
    model, trace = pickle.load(open('SEIR.pickled', 'rb'))

INFO: This code is not longer developed, use our new module instead: github.com/Priesemann-Group/covid19_inference
pr_beta_I_begin was set to default value 100
pr_beta_E_begin_scale was set to default value 10
pr_median_lambda_0 was set to default value 2
pr_sigma_lambda_0 was set to default value 0.7
pr_median_mu was set to default value 0.3333333333333333
pr_sigma_mu was set to default value 0.3
pr_median_delay was set to default value 5
pr_sigma_delay was set to default value 0.2
scale_delay was set to default value 0.3
pr_beta_sigma_obs was set to default value 10
pr_mean_median_incubation was set to default value 5
pr_sigma_median_incubation was set to default value 1
sigma_incubation was set to default value 0.418
week_end_days was set to default value (6, 7)
pr_mean_weekend_factor was set to default value 0.7
pr_sigma_weekend_factor was set to default value 0.17
pr_sigma_date_begin_transient of change point 0 was set to default value 3
pr_median_transient_len of change point 0 w

Auto-assigning NUTS sampler...
Initializing NUTS using advi+adapt_diag...
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
Average Loss = 461.44:   9%|▊         | 17494/200000 [03:06<32:21, 93.98it/s]
Convergence achieved at 17500
Interrupted at 17,499 [8%]: Average Loss = 539.2
Sequential sampling (2 chains in 1 job)
NUTS: [offset_modulation_rad, weekend_factor, median_incubation, sigma_obs, delay, mu, transient_len_2, transient_len_1, transient_len_0, transient_begin_2, transient_begin_1, transient_begin_0, lambda_3, lambda_2, lambda_1, lambda_0, E_begin, E_begin_scale, I_begin]
100%|██████████| 5000/5000 [2:05:22<00:00,  1.50s/it]
There was 1 divergence after tuning. Increase `

FileNotFoundError: ignored

In [2]:
!pip install dvc

In [0]:
#exec(open('figures_revised.py').read())
#matrix = pickle.loads(
import dvc.api
import requests


r = requests.get('https://github.com/ichironakamoto/Japan/blob/master/figures_revised.py')
exec('r')


#create_figure_distributions(model, trace,
#                            additional_insets = {'E_begin_scale': r"$E_0 \simeq ",
#                                                 'median_incubation': r"$D_{inc} \simeq "}, 
#                            xlim_lambda = (0, 4), color = 'tab:purple',
#                            num_changepoints=3, save_to=path_to_save + 'Fig_SEIR_distr')
#create_figure_timeseries(trace, 'tab:purple', num_days_futu_to_plot=21, y_lim_lambda = (-0.5, 4),
#                       plot_red_axis=False, save_to=path_to_save + 'Fig_SEIR_cases')


NameError: ignored

In [34]:
print(pm.loo(trace, model))

NameError: ignored

In [35]:
for j in range(4):
    print(f'lambda* {j}')   
    print(print_median_CI(trace[f"lambda_{j}"] - trace.mu, prec=2))

lambda* 0


NameError: ignored